## Trying Gradio with local llama3

In [ ]:
!pip install langchain langchain-community langchain-core
!pip install tavily-python
!pip install streamlit gradio


In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_19700\706940382.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [2]:
from dotenv import load_dotenv
import os
import getpass
from langchain_community.tools.tavily_search import TavilySearchResults


# Load environment variables from .env file
load_dotenv()

# Access the keys
groq_key = os.getenv("GROQ_API_KEY")
TAVILY_KEY = os.getenv("TAVILY_API_KEY")

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass(groq_key)

os.environ["TAVILY_API_KEY"] = TAVILY_KEY  
search_tool = TavilySearchResults(k=2)

C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_19700\3889205705.py:18: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(k=2)


In [3]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    memory=memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
)


C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_19700\3147404470.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_19700\3147404470.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>

In [ ]:
import gradio as gr

def chat_with_agent(message, history):
    response = agent.run(message)
    return response

gr.ChatInterface(fn=chat_with_agent, title="🦙 LLaMA 3 Agent", chatbot=gr.Chatbot()).launch(inline=True)
### Too slow


## llama3 using Groq API + react_agent from langgraph

In [ ]:
!pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite langchain-openai

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key,
    model="llama3-70b-8192"
)

In [6]:
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()
search = TavilySearch(max_results=2)
tools = [search]

agent_executor = create_react_agent(llm, tools, checkpointer=memory)


In [24]:
config = {"configurable": {"thread_id": "abc123"}}


input_message = {"role": "user", "content": "Hi, I'm Youssef!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Youssef!
================================== Ai Message ==================================

Hi Youssef! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?


In [25]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Youssef!


In [26]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I'm not aware of your name, as we just started our conversation. I don't have any prior information about you. Would you like to introduce yourself?


### Gradio

In [7]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    memory=memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
)


In [ ]:
import gradio as gr

def chat_with_agent(message, history):
    response = agent.run(message)
    return response

gr.ChatInterface(fn=chat_with_agent, title="🦙 LLaMA 3 Agent", chatbot=gr.Chatbot()).launch(inline=True)
